# Исследование прогноза индекса цен на топочный мазут

## Периметр и данные

В исследовании использовались следующие данные:
* Индекс цен на топочный мазут (MZT) на бирже Санкт-Петербурга
* Индекс цен на нефть марки Brent
* Курс доллара США к рублю по ЦБ РФ
* Среднесуточная температура в городах Санкт-Петербург, Костомукша и Петрозаводск

Данные были разделены на обучающую и тестовую выборку
* обучающая выборка с 01.01.2012 до 31.03.2020
* тестовая выборка с 01.04.2020 по 26.08.2021

In [352]:
import numpy as np 
import pandas as pd 
import datetime
from pylab import rcParams
import matplotlib.pyplot as plt
import warnings
import math
import os
from sklearn.metrics import mean_absolute_error
import seaborn as sns
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_percentage_error, f1_score, precision_score, recall_score, confusion_matrix
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
output_notebook()

Loading BokehJS ...

In [37]:
data = pd.read_csv("../../data.csv", sep=";", parse_dates=True)
data = data.drop(["trades", "volume"], axis=1)
data.date = pd.to_datetime(data.date)
data.set_index("date", inplace=True)

## Исходный датасет
сведенный датасет для обучения модели выглядит следующим образом

In [38]:
data.head()

,index_value,oil_price,forex,temp_spb,temp_kost,temp_petr,day_of_week,day_of_year,day_of_month,year,month,week,is_month_end,is_month_start
date,,,,,,,,,,,,,,
2012-01-12,11124.0,112.9700,30.811000,-0.4,-6.400000,-4.1,3,12,12,2012,1,2,0,0
2012-01-13,11124.0,109.8800,31.680700,-0.1,-5.700000,-1.8,4,13,13,2012,1,2,0,0
2012-01-16,11124.0,110.3825,31.870975,-3.2,-8.000000,-3.3,0,16,16,2012,1,3,0,0
2012-01-17,11124.0,110.5500,31.934400,-4.3,-6.500000,-4.1,1,17,17,2012,1,3,0,0
2012-01-18,11124.0,109.8100,31.544500,-7.3,-7.016667,-8.3,2,18,18,2012,1,3,0,0


## Разведывательный анализ данных

Посмотрим на общую динамику временных рядов и верхнеуровневые зависимости между ними

In [363]:
source = ColumnDataSource(data.reset_index())

In [364]:
def plot_ts(ts:str, title:str, line_color:str, line_width:int=2):
    hover=HoverTool(
        tooltips=[
            ('Дата', "@date{%F}"),
            ('Значение', '@{'+ts+'}{0.2f}'),
        ],
        formatters={
            '@date': 'datetime',
            f'{ts}': 'printf',
        },
        mode='vline'
    )
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
    p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title=title)
    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Дата'
    p.yaxis.axis_label = ts
    p.axis.axis_line_color = None
    p.line(y=ts, x="date", source=source, line_width=line_width, line_color=line_color)
    p.add_tools(hover)
    show(p)

In [365]:
# TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
# p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title="Изменение цены на мазут с 2012 по 2021 гг")
# p.extra_y_ranges = {"foo": Range1d(start=0, end=130)}
# p.add_layout(LinearAxis(y_range_name="foo"), 'right')
# p.line(y="index_value", x="date", source=source, line_width=2, line_color="navy")
# p.line(y="oil_price", x="date", source=source, y_range_name="foo", line_color="orange")
# show(p)

#### Мазут

In [366]:
plot_ts("index_value", "Изменение цены на мазут с 2012 по 2021 гг", "navy")

#### Нефть

In [367]:
plot_ts("oil_price", "Изменение цены на нефть с 2012 по 2021 гг", "orange")

#### Курс доллара

In [368]:
plot_ts("forex", "Изменение курса доллара с 2012 по 2021 гг", "darkgreen", 0.5)

In [369]:
def plot_weather():
    hover=HoverTool(
        tooltips=[
            ('Дата', "@date{%F}"),
            ('Cанкт-Петербург', '@{temp_spb}{0.2f}°'),
            ('Костомукша', '@{temp_kost}{0.2f}°'),
            ('Петрозаводск', '@{temp_petr}{0.2f}°'),
        ],
        formatters={
            '@date': 'datetime',
            'temp_spb': 'printf',
            'temp_kost': 'printf',
            'temp_petr': 'printf',
        },
        mode='vline'
    )
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
    p = figure(x_axis_type='datetime', width=1000, height=300, tools=TOOLS, title="Температура")
    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Дата'
    p.yaxis.axis_label = "Температура"
    p.axis.axis_line_color = None
    p.line(y="temp_spb", x="date", source=source, line_width=0.5, line_color="blue")
    p.line(y="temp_kost", x="date", source=source, line_width=0.5, line_color="red")
    p.line(y="temp_petr", x="date", source=source, line_width=0.5, line_color="green")
    p.add_tools(hover)
    show(p)

#### Погода в Санкт-Петербурге, Костомукше и Петрозаводске

In [117]:
plot_weather()

In [378]:
def mzt_and_oil():
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
    hover=HoverTool(
        tooltips=[
            ('Дата', "@date{%F}"),
            ('Цена на мазут', '@{index_value}{0.2f} руб'),
            ('Цена на нефть', '$@{oil_price}{0.2f}'),
        ],
        formatters={
            '@date': 'datetime',
            f'index_value': 'printf',
            f'oil_price': 'printf',
        },
#         mode='vline'
    )
    p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title="Совместная динамика цен на нефть и мазут с 2012 по 2021 гг")
    p.extra_y_ranges = {"foo": Range1d(start=0, end=130)}
    p.add_layout(LinearAxis(y_range_name="foo"), 'right')
    p.add_tools(hover)
    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Дата'
    p.yaxis.axis_label = "Цена"
    p.axis.axis_line_color = None
    p.line(y="index_value", x="date", source=source, line_width=2, line_color="navy", legend_label="Мазут")
    p.line(y="oil_price", x="date", source=source, y_range_name="foo", line_color="orange", legend_label="Нефть")
    breakpoints = [
        datetime.datetime.strptime("01-07-2014", "%d-%m-%Y"),
        datetime.datetime.strptime("01-01-2015", "%d-%m-%Y"),
        datetime.datetime.strptime("01-05-2020", "%d-%m-%Y"),
        
    ]
    p.line(x=breakpoints[0], y=[0, 30000], line_color="grey", line_alpha=0.5, line_width=2, line_dash="dotted")
    p.line(x=breakpoints[1], y=[0, 30000], line_color="grey", line_alpha=0.5, line_width=2, line_dash="dotted")
    p.line(x=breakpoints[2], y=[0, 30000], line_color="grey", line_alpha=0.5, line_width=2, line_dash="dotted")
    p.legend.location="bottom_left"

    show(p)

#### Наложим цены на мазут и на нефть на один график

In [379]:
mzt_and_oil()

### Корреляция с ценой на нефть

На графике выше невооруженным глазом заметна корреляция между ценами на мазут и ценами на нефть.
Однако так же видно, что резкое падение цен на нефть в июле 2014 года, которое продолжалось до начала 2015 года, не значительно повлияло на цены на мазут

Сравним корреляцию на разных отрезках времени

In [385]:
print(f'Корреляция на всем временном промежутке: {data.corrwith(data.index_value)["oil_price"]:.2%}')
print(f'Корреляция с 1 января 2015 по 1 мая 2020: {data["01-01-2015":"01-05-2020"].corrwith(data.index_value)["oil_price"]:.2%}')
print(f'Корреляция с 1 июля 2014 по 1 января 2015: {data["01-07-2014":"01-01-2015"].corrwith(data.index_value)["oil_price"]:.2%}')
print(f'Корреляция до 1 июля 2014: {data[:"01-07-2014"].corrwith(data.index_value)["oil_price"]:.2%}')
print(f'Корреляция c 1 мая 2020: {data["01-05-2020":].corrwith(data.index_value)["oil_price"]:.2%}')

Корреляция на всем временном промежутке: 7.93%
Корреляция с 1 января 2015 по 1 мая 2020: 83.49%
Корреляция с 1 июля 2014 по 1 января 2015: 58.34%
Корреляция до 1 июля 2014: 23.06%
Корреляция c 1 мая 2020: 86.61%


Видно, что на отдельных интервалах корреляция очень большая (больше 86%, начиная с мая 2020 года), однако на всем промежутке всего 7.93% <br>
Попробуем визуализировать эту зависимость

**NB**: чем более вытянута в линию последовательность точек, тем выше сила корреляции. И наоборот, чем больше группа точек похожа на облако, тем корреляция ниже

In [216]:
oil_mzt_corr = data.copy()
oil_mzt_corr["marker"]=""
oil_mzt_corr[:"01-07-2014"]["marker"] = "before jul 2014"
oil_mzt_corr["01-07-2014":"01-01-2015"]["marker"] = "between jul 2014 & jan 2015"
oil_mzt_corr["01-01-2015":"01-05-2020"]["marker"] = "between jan 2015 & may 2020"
oil_mzt_corr["01-05-2020":]["marker"] = "after may 2020"
om_source = ColumnDataSource(oil_mzt_corr)
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
hover=HoverTool(
    tooltips=[
        ('Дата', "@date{%F}"),
        ('Цена на мазут', '@{index_value}{0.2f} руб'),
        ('Цена на нефть', '$@{oil_price}{0.2f}'),
        ('Интервал', '@{marker}')
    ],
    formatters={
        '@date': 'datetime',
        f'index_value': 'printf',
        f'oil_price': 'printf',
    },
#         mode='vline'
)

p = figure( width=700, height=700, tools=TOOLS, title="Корреляция цен на мазут и нефть на разных интервалах")
p.add_tools(hover)
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.xaxis.axis_label = 'Цена на мазут'
p.yaxis.axis_label = "Цена на нефть"
p.axis.axis_line_color = None
index_cmap = factor_cmap('marker', palette=['#f16a70', '#4d4d4d', '#b1d877', '#8cdcda'], 
                         factors=sorted(oil_mzt_corr.marker.unique()))
p.scatter("index_value", "oil_price", color=index_cmap, source=om_source, size=6, alpha=0.65)
show(p)

## Корреляция с курсом доллара
Совместная динамика с курсом доллара не такая заметная, как с нефтью </br>

In [384]:
print(f'Корреляция на всем временном промежутке: {data.corrwith(data.index_value)["forex"]:.2%}')

Корреляция на всем временном промежутке: 29.89%


In [386]:
def mzt_and_forex():
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
    hover=HoverTool(
        tooltips=[
            ('Дата', "@date{%F}"),
            ('Цена на мазут', '@{index_value}{0.2f} руб'),
            ('Курс доллара', '@{forex}{0.2f}'),
        ],
        formatters={
            '@date': 'datetime',
            f'index_value': 'printf',
            f'forex': 'printf',
        },
#         mode='vline'
    )
    p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title="Совместная динамика цен на мазут и курс доллара с 2012 по 2021 гг")
    p.extra_y_ranges = {"foo": Range1d(start=0, end=100)}
    p.add_layout(LinearAxis(y_range_name="foo"), 'right')
    p.add_tools(hover)
    p.background_fill_color="#f5f5f5"
    p.grid.grid_line_color="white"
    p.xaxis.axis_label = 'Дата'
    p.yaxis.axis_label = "Цена"
    p.axis.axis_line_color = None
    p.line(y="index_value", x="date", source=source, line_width=2, line_color="navy", legend_label="Мазут")
    p.line(y="forex", x="date", source=source, y_range_name="foo", line_color="olive", legend_label="Курс доллара")
    breakpoints = [
        datetime.datetime.strptime("01-01-2015", "%d-%m-%Y"),
        
    ]
    p.line(x=breakpoints[0], y=[0, 30000], line_color="grey", line_alpha=0.5, line_width=2, line_dash="dotted")
    p.legend.location="top_left"
    show(p)

In [387]:
mzt_and_forex()

In [235]:
frx_mzt_corr = data.copy()
frx_mzt_corr["marker"]=""
frx_mzt_corr[:"01-05-2015":]["marker"] = "before may 2015"
frx_mzt_corr["01-05-2015":]["marker"] = "after may 2021"
fr_source = ColumnDataSource(frx_mzt_corr)
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
hover=HoverTool(
    tooltips=[
        ('Дата', "@date{%F}"),
        ('Цена на мазут', '@{index_value}{0.2f} руб'),
        ('Курс доллара', '$@{forex}{0.2f}'),
    ],
    formatters={
        '@date': 'datetime',
        f'index_value': 'printf',
        f'forex': 'printf',
    },
#         mode='vline'
)

p = figure( width=700, height=700, tools=TOOLS, title="Корреляция цен на мазут и курс доллара с 2012 по 2021 гг")
p.add_tools(hover)
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.xaxis.axis_label = 'Цена на мазут'
p.yaxis.axis_label = "Курс доллара"
p.axis.axis_line_color = None
index_cmap = factor_cmap('marker', palette=['#b1d877', '#8cdcda'], 
                         factors=sorted(frx_mzt_corr.marker.unique()))
p.scatter("index_value", "forex", color=index_cmap, source=fr_source, size=6, alpha=0.65)
show(p)

## Корреляция с другими переменными

In [313]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'

corr_source = pd.DataFrame(data["01-01-2015":].corrwith(data.index_value)).reset_index()
corr_source.columns = ['var', 'val']
corr_source = corr_source[corr_source['var']!="index_value"]
corr_source.sort_values(by="val", ascending=False, inplace=True)
cats = corr_source["var"].unique()
corr_source = ColumnDataSource(corr_source)\

hover=HoverTool(
    tooltips=[
        ('Переменная', "@var"),
        ('Корреляция', '@{val}{0.2%}'),
    ],
    formatters={
        '@val': 'printf',
    },
#         mode='vline'
)

p = figure( width=700, height=400, tools=TOOLS, title="Корреляция цен на мазут c прочими переменными с 2015 по 2021 гг", x_range=cats)
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.axis.axis_line_color = None
p.add_tools(hover)
p.vbar(x="var", top="val", source=corr_source, width=0.8)
p.xaxis.major_label_orientation = math.pi/4
show(p)

## Прогноз будущих значений

Для прогноза будущих значений цен на мазут были испробованы следующие алгоритмы (в порядке увеличения сложности модели):
* Авторегрессионные модели (SARIMAX)
* fbprophet (Holt Winters)
* Градиентный бустинг LightGBM c последовательной валидацией
* Градиентный бустинг LightGBM с прямым построением моделей на горизонт планирования
* Нейронная сеть LSTM c последовательной валидацией
* Нейронная сеть LSTM c прогнозом последовательностей
* Энкодер-декодер LSTM сеть
* Энкодер-декодер LSTM сеть с механизмом внимания

В связи с тем, что захватить какие-то закономерности получилось только у нейронной сети с механизмом внимания, то остановимся на ней

### Подход со скользящим окном

В постановке задачи было указано, что необходимо прогнозировать цены на 1 месяц вперед. Для этого мы будем использовать известные данные за предыдущие 3 месяца, которые наиболее сильно влияют на динамику цен. 

![title](approach.png)

### LSTM
В то же время архитектура LSTM (long-short term memory) и механизм внимания позволяют нам учитывать и закономерности, которые были выявлены в предыдущих периодах

![title](model.png)

In [337]:
preds = pd.read_csv("pred.csv")
preds["error"] = np.abs(preds.ytrue - preds.yhat) / preds.ytrue
preds.date = pd.to_datetime(preds.date)

### Визуализация прогноза

In [338]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
hover=HoverTool(
    tooltips=[
        ("Дата", "@date{%F}"),
        ('Реальное значение', "@{ytrue}{0.00}"),
        ('Прогноз', '@{yhat}{0.00}'),
        ('Ошибка', '@{error}{0.2%}'),
    ],
    formatters={
        '@date': 'datetime',
        '@ytrue': 'printf',
        '@yhat': 'printf',
        '@error': 'printf',
    },
        mode='vline',
    names=["yhat"]
)
pr_source = ColumnDataSource(preds)
p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title="Прогноз цен на мазут с использованием нейронной сети")
p.line("date", "ytrue", source=pr_source, legend_label="Реальные значения", line_width=2, line_color="navy")
p.line("date", "yhat", source=pr_source, legend_label="Прогноз", line_width=2, line_color="red", name="yhat")
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.axis.axis_line_color = None
p.add_tools(hover)

p.legend.location="top_left"
show(p)

Как видно, модель способна только угадывать *тренд роста цен*, но *не флуктуации внутри месяца*. В связи с этим, использовать данную модель для принятия решений о закупке в определенный день **не рекомендуется**

Однако модель может быть полезной для анализа и мониторинга общих тенденций и трендов, не учитывая при этом общемировые и политические события, которые определяют изменение цен на нефть

## What if? Прогноз цен на мазут при известной цене на нефть.

Гипотеза заключается в том, чтобы проверить возможность объяснить разброс (дисперсию) цен на мазут, если цена на нефть уже известна. Возможно, если найти какие-то более-менее достоверные прогнозы цен на нефть с достаточной гранулярностью (день, неделя), то объяснить цены на мазут станет возможным

In [347]:
proph_df = pd.read_csv("prophet_oil_forex.csv")
proph_df.date = pd.to_datetime(proph_df.date)
ph_source = ColumnDataSource(proph_df)

In [349]:
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom'
hover=HoverTool(
    tooltips=[
        ("Дата", "@date{%F}"),
        ('Реальное значение', "@{y_true}{0.00}"),
        ('Прогноз', '@{y_hat}{0.00}'),
        ('Ошибка', '@{error}{0.2%}'),
    ],
    formatters={
        '@date': 'datetime',
        '@y_true': 'printf',
        '@y_hat': 'printf',
        '@error': 'printf',
    },
        mode='vline',
    names=["yhat"]
)
p = figure(x_axis_type='datetime', width=1000, height=400, tools=TOOLS, title="Прогноз цен на мазут с использованием нейронной сети")
p.line("date", "y_true", source=ph_source, legend_label="Реальные значения", line_width=2, line_color="navy")
p.line("date", "y_hat", source=ph_source, legend_label="Прогноз", line_width=2, line_color="red", name="yhat")
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.axis.axis_line_color = None
p.add_tools(hover)

p.legend.location="top_left"
show(p)

По графику видно, что даже если есть ошибки, и расстояние между прогнозом и реальным значением может быть велико, в целом прогноз способен повторять пики и падения цен. Поэтому стоит попробовать решить прикладную задачу по рекомендации покупки

Так как прогноз Top-5, Bottom-5 показал плохие результаты, упростим задачу:
* На горизонте планирования считаем медианную цену (ту, ниже которой находится ровно половина наблюдений за этот период)
* Каждому значению в реальных значениях и прогнозе поставим флажок
* Если значение меньше медианы, то флажок - 0, можно покупать
* Если значение выше медианы, то флажок - 1, стоит подождать

In [350]:
Y_TRUE_MEDIAN = proph_df.y_true.median()
Y_HAT_MEDIAN = proph_df.y_hat.median()
proph_df["true_hold"] = proph_df.y_true.apply(lambda x: 1 if x >= Y_TRUE_MEDIAN else 0)
proph_df["hat_hold"] = proph_df.y_hat.apply(lambda x: 1 if x >= Y_HAT_MEDIAN else 0)    

In [357]:
pd.DataFrame(confusion_matrix(proph_df.true_hold, proph_df.hat_hold), columns=["Прогноз покупать", "Прогноз ждать"], index=["Факт покупать", "Факт ждать"])

,Прогноз покупать,Прогноз ждать
Факт покупать,30,9
Факт ждать,9,31


In [358]:
print(f"Взвешенная метрика точности модели (F1) = {f1_score(proph_df.true_hold, proph_df.hat_hold):.2%}")

Взвешенная метрика точности модели (F1) = 77.50%


## Выводы

* Цена на мазут очень сильно зависит от мировых цен на нефть
* В связи с тем, что цена на нефть определяется в большей степенью общемировыми и политическими событиями, достоверно предсказывать ее на исторических данных не кажется возможным
* Сложные модели глубокого обучения позволяют в какой-то степени определить тренд и закономерности на среднесрочном горизонте планирования
* Точности модели недостаточно для эффективных рекомендаций по покупке мазут
* При наличии прогноза цен на нефть с достаточной гранулярностью (день, неделя) можно добиться приемлимой точности для использования результатов в рекомендациях о покупке